In [1]:
from PIL import Image
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, LeakyReLU, Conv2D, Conv2DTranspose, Add, Lambda, Flatten, Reshape
from keras.optimizers import Adam
from tensorflow.keras.layers import PReLU
from keras.regularizers import l2
import keras.backend as K
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import heapq
import collections
from typing import List, Dict, Tuple
import tensorflow as tf

## Resize, conversion and normalisation des photos



Fonction de resize, normalisation, conversion en nuances de gris


In [2]:
def resize_and_convert_to_gray(input_path, size=(128, 128)):
    with Image.open(input_path) as img:
        img = img.resize(size)
        gray_img = img.convert('L')
        gray_img.save("Image_grise_1.png")
        img_array = np.asarray(gray_img, dtype=np.float32) / 255.0
        return img_array.reshape(size[0], size[1], 1)

In [4]:
def convert_to_gray_and_save(img_array, output_path):

    img_array *= 255.0
    img_array = img_array.clip(0, 255)
    img_gray = Image.fromarray(np.uint8(img_array), 'L')
    img_gray.save(output_path)

    return img_gray


## Autoencoder

Définition de l'autoencoder

In [5]:
# Taille de l'image d'entrée
input_shape = (128, 128, 1)

# Encoder
encoder_input = Input(shape=input_shape, name="encoder_input")
x = Conv2D(32, (3, 3), padding='same')(encoder_input)
x = LeakyReLU()(x)
x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
x = LeakyReLU()(x)
x = Flatten()(x)
encoder_output = Dense(units=2, name="encoder_output")(x)  # Suppose the latent space has 2 dimensions

encoder = Model(encoder_input, encoder_output, name="encoder_model")
encoder.summary()

Model: "encoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 128)     

Définition du decoder

In [6]:
# Decoder
decoder_input = Input(shape=(2,), name="decoder_input")  # Latent space dimensions
x = Dense(units=32 * 32 * 128)(decoder_input)  # Adjust the units to match the encoder's last Conv2D output shape
x = LeakyReLU()(x)
x = Reshape((32, 32, 128))(x)
x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x)
x = LeakyReLU()(x)
x = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(x)
x = LeakyReLU()(x)
decoder_output = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x) # Use sigmoid for normalizing the output between 0 and 1

decoder = Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 131072)            393216    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 131072)            0         
                                                                 
 reshape (Reshape)           (None, 32, 32, 128)       0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 64, 64, 64)        73792     
 anspose)                                                        
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                     

Création de l'autoencoder à partir de l'encoder et du decoder

In [7]:
ae_input = Input(shape=input_shape, name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

autoencoder = Model(ae_input, ae_decoder_output, name="AE")
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
autoencoder.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AE_input (InputLayer)       [(None, 128, 128, 1)]     0         
                                                                 
 encoder_model (Functional)  (None, 2)                 354818    
                                                                 
 decoder_model (Functional)  (None, 128, 128, 1)       485761    
                                                                 
Total params: 840579 (3.21 MB)
Trainable params: 840579 (3.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
#autoencoder = load_model("autoencoder_model10.keras", safe_mode=False)

Résumé de l'autoencoder

In [8]:
autoencoder.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AE_input (InputLayer)       [(None, 128, 128, 1)]     0         
                                                                 
 encoder_model (Functional)  (None, 2)                 354818    
                                                                 
 decoder_model (Functional)  (None, 128, 128, 1)       485761    
                                                                 
Total params: 840579 (3.21 MB)
Trainable params: 840579 (3.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Création des dataset de train et de test. 80-20%

In [11]:
image_files = glob.glob('Kodak/*.png')
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

Entrainement et sauvegarde du modèle

In [ ]:
# Charger et préparer les images
train_images = np.array([resize_and_convert_to_gray(path) for path in train_files])
test_images = np.array([resize_and_convert_to_gray(path) for path in test_files])

# Entraîner l'autoencoder
autoencoder.fit(train_images, train_images, epochs=50000, batch_size=16, validation_data=(test_images, test_images))
autoencoder.save('autoencoder_model10.keras')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2/2 [==============================] - 0s 31ms/step - loss: 2.8643e-05 - val_loss: 0.0636
Epoch 28172/50000
2/2 [==============================] - 0s 30ms/step - loss: 2.6834e-05 - val_loss: 0.0638
Epoch 28173/50000
2/2 [==============================] - 0s 30ms/step - loss: 3.0636e-05 - val_loss: 0.0638
Epoch 28174/50000
2/2 [==============================] - 0s 31ms/step - loss: 2.9361e-05 - val_loss: 0.0632
Epoch 28175/50000
2/2 [==============================] - 0s 37ms/step - loss: 2.9858e-05 - val_loss: 0.0634
Epoch 28176/50000
2/2 [==============================] - 0s 35ms/step - loss: 2.7665e-05 - val_loss: 0.0642
Epoch 28177/50000
2/2 [==============================] - 0s 29ms/step - loss: 2.9117e-05 - val_loss: 0.0638
Epoch 28178/50000
2/2 [==============================] - 0s 29ms/step - loss: 2.6779e-05 - val_loss: 0.0633
Epoch 28179/50000
2/2 [==============================] - 0s 29ms/step - loss

## Test de reconstruction d'images pour le test

---



In [ ]:
image = resize_and_convert_to_gray('Kodak/kodim12.png')
image= np.expand_dims(image, axis=0)
reconstructed_images = autoencoder.predict(image)
output_folder = 'compressed_images/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, img_array in enumerate(reconstructed_images):
    img = Image.fromarray((img_array.squeeze() * 255).astype(np.uint8))
    img.save(os.path.join(output_folder, f'image_compressee_{i+1:02d}.png'))

## Fonctions

In [ ]:
def encoding(image_path, enc):
  image = resize_and_convert_to_gray(image_path)
  image = np.expand_dims(image, axis=0)
  encoded_image = enc.predict(image)
  #encoded_vector = encoded_image[0]
  #print(encoded_vector)
  #return encoded_vector
  return encoded_image

In [ ]:
def decoding(output_path,dec, y):
  print(y)
  #encoded_vector_batch = np.expand_dims(y, axis=0)
  #reconstructed_data = dec.predict(encoded_vector_batch)
  reconstructed_data = dec.predict(y)
  image_finale =  reconstructed_data[0]
  #print(image_finale)
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  img = Image.fromarray((image_finale.squeeze() * 255).astype(np.uint8))
  img.save(os.path.join(output_path, f'image_compressee.png'))
  #convert_to_gray_and_save(image_finale.squeeze(), output_path)

## Simulation

### Encodage

In [ ]:
encoded_vector = encoding('Kodak/kodim04.png', encoder)

Test

In [ ]:
encoded_vector.shape

In [ ]:
encoded_vector.dtype

### Decodage

In [ ]:
decoding("image_test/",decoder, encoded_vector)

In [ ]:
print(encoded_vector)